In [4]:
import speech_recognition as sr
from gtts import gTTS
from IPython.display import Audio, display, clear_output
import sqlite3
import datetime
import time
import os
import random

# --- 1. CONFIGURATION & DATABASE ---
DB_PATH = "simplotel_notebook.db"

def init_db():
    """Initialize SQLite database for analytics"""
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS interactions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            speaker TEXT,
            message TEXT,
            intent TEXT
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS bookings (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            room_type TEXT,
            status TEXT
        )
    """)
    conn.commit()
    conn.close()

init_db()

# --- 2. BACKEND (The Hotel System) ---
class HotelBackend:
    def __init__(self):
        # Simulating Hotel Inventory
        self.inventory = {
            "standard": {"price": 4000, "qty": 5},
            "deluxe": {"price": 7500, "qty": 2},
            "suite": {"price": 15000, "qty": 1}
        }

    def check_price(self, room_type):
        if room_type in self.inventory:
            return self.inventory[room_type]["price"]
        return None

    def book_room(self, room_type):
        if room_type in self.inventory and self.inventory[room_type]["qty"] > 0:
            self.inventory[room_type]["qty"] -= 1
            return True
        return False

# --- 3. ANALYTICS ENGINE ---
class Analytics:
    def log_chat(self, speaker, message, intent=""):
        conn = sqlite3.connect(DB_PATH)
        ts = datetime.datetime.now().strftime("%H:%M:%S")
        conn.execute("INSERT INTO interactions (timestamp, speaker, message, intent) VALUES (?, ?, ?, ?)", 
                     (ts, speaker, message, intent))
        conn.commit()
        conn.close()

    def log_booking(self, room_type, status):
        conn = sqlite3.connect(DB_PATH)
        ts = datetime.datetime.now().strftime("%Y-%m-%d")
        conn.execute("INSERT INTO bookings (timestamp, room_type, status) VALUES (?, ?, ?)", 
                     (ts, room_type, status))
        conn.commit()
        conn.close()

    def show_report(self):
        conn = sqlite3.connect(DB_PATH)
        print("\n" + "="*60)
        print("📊 SESSION ANALYTICS REPORT")
        print("="*60)
        
        # Fetch last 10 interactions
        logs = conn.execute("SELECT timestamp, speaker, message FROM interactions ORDER BY id DESC LIMIT 10").fetchall()
        for log in reversed(logs):
            print(f"[{log[0]}] {log[1].upper()}: {log[2]}")
            
        print("-" * 60)
        # Fetch Bookings
        bookings = conn.execute("SELECT * FROM bookings ORDER BY id DESC LIMIT 5").fetchall()
        if bookings:
            print(f"✅ Total Bookings Created: {len(bookings)}")
        else:
            print("⚠️ No bookings made in this session.")
        print("="*60)
        conn.close()

# --- 4. THE VOICE BOT (Brain) ---
class SimplotelBot:
    def __init__(self):
        self.backend = HotelBackend()
        self.analytics = Analytics()
        self.recognizer = sr.Recognizer()
        self.is_running = True

    def speak(self, text):
        """
        1. Prints text.
        2. Generates MP3.
        3. Plays Audio in Notebook.
        """
        # Visual Output
        print(f"🤖 AI: {text}")
        self.analytics.log_chat("AI", text)

        # Audio Output (Google TTS)
        try:
            tts = gTTS(text=text, lang='en')
            filename = f"response_{random.randint(1,1000)}.mp3"
            tts.save(filename)
            
            # Auto-play the audio
            display(Audio(filename, autoplay=True))
            
            # Wait duration roughly equal to speech length to prevent self-listening
            time.sleep(len(text) / 12)  
            
            # Cleanup file
            os.remove(filename)
        except Exception as e:
            print(f"(Audio Error: {e})")

    def listen(self):
        with sr.Microphone() as source:
            print("\n🎤 Listening... (Speak now)")
            self.recognizer.adjust_for_ambient_noise(source, duration=0.5)
            try:
                audio = self.recognizer.listen(source, timeout=5)
                text = self.recognizer.recognize_google(audio).lower()
                print(f"👤 You: {text}")
                self.analytics.log_chat("User", text)
                return text
            except sr.WaitTimeoutError:
                return None
            except sr.UnknownValueError:
                return None
            except:
                return None

    def process_input(self, text):
        if not text:
            return
        
        text = text.lower()

        # --- INTENT LOGIC ---
        
        # 1. Exit
        if any(w in text for w in ["bye", "exit", "stop", "goodbye"]):
            self.speak("Goodbye! Thank you for visiting Simplotel.")
            self.is_running = False
            return

        # 2. Greeting
        if any(w in text for w in ["hi", "hello", "hey"]):
            self.speak("Welcome to Simplotel Hotel Assistant. I can help you book a room or check prices.")
            return

        # 3. Check Prices
        if "price" in text or "cost" in text or "rate" in text:
            self.speak("Standard rooms are 4000, Deluxe are 7500, and Suites are 15000 rupees.")
            return

        # 4. Booking Logic
        if "book" in text:
            # Extract Room Entity
            room_type = None
            if "suite" in text: room_type = "suite"
            elif "deluxe" in text: room_type = "deluxe"
            elif "standard" in text: room_type = "standard"

            if room_type:
                # Check Inventory
                if self.backend.book_room(room_type):
                    self.analytics.log_booking(room_type, "Confirmed")
                    self.speak(f"Success! I have booked a {room_type} room for you.")
                else:
                    self.analytics.log_booking(room_type, "Failed")
                    self.speak(f"Sorry, the {room_type} rooms are currently sold out.")
            else:
                self.speak("Which room would you like to book? We have Standard, Deluxe, and Suite.")
            return

        # 5. Fallback
        self.speak("I didn't quite get that. Try saying 'Check prices' or 'Book a Deluxe room'.")

    def start(self):
        # Initial greeting
        self.speak("System Online. How can I help you?")
        
        while self.is_running:
            user_input = self.listen()
            if user_input:
                self.process_input(user_input)
            else:
                # If silence, just wait a bit and loop
                time.sleep(1)
        
        # Show Analytics on exit
        self.analytics.show_report()

# --- 5. RUNNER ---
# Create and run the bot
bot = SimplotelBot()
bot.start()

🤖 AI: System Online. How can I help you?



🎤 Listening... (Speak now)
👤 You: hello can you help me for book a room
🤖 AI: Welcome to Simplotel Hotel Assistant. I can help you book a room or check prices.



🎤 Listening... (Speak now)
👤 You: so tell me about the prices of your room
🤖 AI: Standard rooms are 4000, Deluxe are 7500, and Suites are 15000 rupees.



🎤 Listening... (Speak now)
👤 You: cool so book a suit for me
🤖 AI: Which room would you like to book? We have Standard, Deluxe, and Suite.



🎤 Listening... (Speak now)
👤 You: book a sweet for me
🤖 AI: Which room would you like to book? We have Standard, Deluxe, and Suite.



🎤 Listening... (Speak now)

🎤 Listening... (Speak now)

🎤 Listening... (Speak now)
👤 You: book a deluxe room for me
🤖 AI: Success! I have booked a deluxe room for you.



🎤 Listening... (Speak now)
👤 You: thank u so mach and good bye
🤖 AI: Goodbye! Thank you for visiting Simplotel.



📊 SESSION ANALYTICS REPORT
[15:15:21] USER: so tell me about the prices of your room
[15:15:21] AI: Standard rooms are 4000, Deluxe are 7500, and Suites are 15000 rupees.
[15:15:38] USER: cool so book a suit for me
[15:15:38] AI: Which room would you like to book? We have Standard, Deluxe, and Suite.
[15:15:53] USER: book a sweet for me
[15:15:53] AI: Which room would you like to book? We have Standard, Deluxe, and Suite.
[15:16:24] USER: book a deluxe room for me
[15:16:24] AI: Success! I have booked a deluxe room for you.
[15:16:33] USER: thank u so mach and good bye
[15:16:33] AI: Goodbye! Thank you for visiting Simplotel.
------------------------------------------------------------
✅ Total Bookings Created: 1
